In [2]:
import copy
import os
import json
import numpy as np
from tqdm import tqdm
from collections import Counter
from argparse import ArgumentParser
from factor_utils import factor_evidence, factor_marginalize, assignment_to_index
from factor import Factor

# taken from part 1
import copy
import numpy as np
from factor import Factor, index_to_assignment, assignment_to_index


def factor_product(A, B):
    """
    Computes the factor product of A and B e.g. A = f(x1, x2); B = f(x1, x3); out=f(x1, x2, x3) = f(x1, x2)f(x1, x3)

    Args:
        A: first Factor
        B: second Factor

    Returns:
        Returns the factor product of A and B
    """
    out = Factor()

    """ YOUR CODE HERE """
    if A.is_empty():
        return B
    if B.is_empty():
        return A
    # Create output factor. Variables should be the union between of the
    # variables contained in the two input factors
    out = Factor()
    out.var = np.union1d(A.var, B.var)

    # Compute mapping between the variable ordering between the two factors
    # and the output to set the cardinality
    out.card = np.zeros(len(out.var), np.int64)
    mapA = np.argmax(out.var[None, :] == A.var[:, None], axis=-1)
    mapB = np.argmax(out.var[None, :] == B.var[:, None], axis=-1)
    out.card[mapA] = A.card
    out.card[mapB] = B.card

    # For each assignment in the output, compute which row of the input factors
    # it comes from
    out.val = np.zeros(np.prod(out.card))
    assignments = out.get_all_assignments()
    idxA = assignment_to_index(assignments[:, mapA], A.card)
    idxB = assignment_to_index(assignments[:, mapB], B.card)

    out.val = np.array(A.val)[idxA] * np.array(B.val)[idxB]
    """ END YOUR CODE HERE """
    return out


def factor_marginalize(factor, var):
    """
    Returns factor after variables in var have been marginalized out.

    Args:
        factor: factor to be marginalized
        var: numpy array of variables to be marginalized over

    Returns:
        marginalized factor
    """
    out = copy.deepcopy(factor)

    """ YOUR CODE HERE
     HINT: Use the code from lab1 """
    out.var = np.setdiff1d(factor.var, var)
    out.card = factor.card[np.isin(factor.var, out.var)]
    out.val = np.zeros(np.prod(out.card))

    unNormVar=np.squeeze(np.take(factor.get_all_assignments(),np.where(~np.isin(factor.var, var)),axis=1),axis=1)
    unNormVarIndex = assignment_to_index(unNormVar, out.card)

    unNormVal= [np.sum(np.array(factor.val)[unNormVarIndex == i]) for i in range(len(out.val))] 
    out.val=unNormVal/np.sum(unNormVal)
    #out.val=unNormVal
    """ END YOUR CODE HERE """
    return out


def factor_evidence(factor, evidence):
    """
    Observes evidence and retains entries containing the observed evidence. Also removes the evidence random variables
    because they are already observed e.g. factor=f(1, 2) and evidence={1: 0} returns f(2) with entries from node1=0
    Args:
        factor: factor to reduce using evidence
        evidence:  dictionary of node:evidence pair where evidence[1] = evidence of node 1.
    Returns:
        Reduced factor that does not contain any variables in the evidence. Return an empty factor if all the
        factor's variables are observed.
    """
    out = copy.deepcopy(factor)

    """ YOUR CODE HERE,     HINT: copy from lab2 part 1! """
    for k in np.intersect1d(out.var,list(evidence.keys())):
            # print("considering: ",k,evidence[k])
            currentVarAssign=np.squeeze(np.take(out.get_all_assignments(),np.where(np.isin(out.var, k )),axis=1),axis=1)
            indexesToChange=np.squeeze(np.where(np.any(currentVarAssign!=evidence[k], axis=1)))
            out.val[indexesToChange]=0.0
    """ END YOUR CODE HERE """

    return out

def compute_joint_distribution(factors):
    """Computes the joint distribution defined by a list of given factors

    Args:
        factors (List[Factor]): List of factors

    Returns:
        Factor containing the joint distribution of the input factor list
    """
    joint = Factor()

    """ YOUR CODE HERE
    Compute the joint distribution from the list of factors. You may assume
    that the input factors are valid so no input checking is required.
    """
    joint = factors[0]
    if len(factors)<2:return joint

    for factor in factors[1:]:
        joint = factor_product(joint, factor)

    return joint

def update_factor_dict_with_evidence(factor, evidence):
    """
    Observes evidence and retains entries containing the observed evidence. Also removes the evidence random variables
    because they are already observed e.g. factor=f(1, 2) and evidence={1: 0} returns f(2) with entries from node1=0
    Args:
        factor: factor to reduce using evidence
        evidence:  dictionary of node:evidence pair where evidence[1] = evidence of node 1.
    Returns:
        Reduced factor that does not contain any variables in the evidence. Return an empty factor if all the
        factor's variables are observed.
    """
    out = copy.deepcopy(factor)

    """ YOUR CODE HERE,     HINT: copy from lab2 part 1! """
    holdingDict={}
    for f in out.items():
        factor=f[1]
        node=f[0]
        listToMarginalize=[]
        for k in np.intersect1d(factor.var,list(evidence.keys())):
            # print("considering: ",k,evidence[k])
            currentVarAssign=np.squeeze(np.take(factor.get_all_assignments(),np.where(np.isin(factor.var, k )),axis=1),axis=1)
            indexesToChange=np.squeeze(np.where(np.any(currentVarAssign!=evidence[k], axis=1)))
            factor.val[indexesToChange]=0.0
            #print(factor,k)
            holdingDict[node]=k
        if listToMarginalize:
            holdingDict[node]=factor_marginalize(factor,listToMarginalize)
        else:
            holdingDict[node]=factor
    out=holdingDict
    """ END YOUR CODE HERE """

    return out

In [3]:
def load_input_file(input_file: str) -> (Factor, dict, dict, int):
    """
    Returns the target factor, proposal factors for each node and evidence. DO NOT EDIT THIS FUNCTION

    Args:
        input_file: input file to open

    Returns:
        Factor of the target factor which is the target joint distribution of all nodes in the Bayesian network
        dictionary of node:Factor pair where Factor is the proposal distribution to sample node observations. Other
                    nodes in the Factor are parent nodes of the node
        dictionary of node:val pair where node is an evidence node while val is the evidence for the node.
    """
    with open(input_file, 'r') as f:
        input_config = json.load(f)
    proposal_factors_dict = input_config['proposal-factors']

    def parse_factor_dict(factor_dict):
        var = np.array(factor_dict['var'])
        card = np.array(factor_dict['card'])
        val = np.array(factor_dict['val'])
        return Factor(var=var, card=card, val=val)

    nodes = np.array(input_config['nodes'], dtype=int)
    edges = np.array(input_config['edges'], dtype=int)
    node_factors = {int(node): parse_factor_dict(factor_dict=proposal_factor_dict) for
                    node, proposal_factor_dict in proposal_factors_dict.items()}

    evidence = {int(node): ev for node, ev in input_config['evidence'].items()}
    initial_samples = {int(node): initial for node, initial in input_config['initial-samples'].items()}

    num_iterations = input_config['num-iterations']
    num_burn_in = input_config['num-burn-in']
    return nodes, edges, node_factors, evidence, initial_samples, num_iterations, num_burn_in

os.getcwd()+"/data"

testcase="2"
nodes, edges, node_factors, evidence, initial_samples, num_iterations, num_burn_in = load_input_file(input_file="/home/jasper/programming/lab4/lab4/part2/data/inputs/"+testcase+".json")
nodes, edges, node_factors, evidence, initial_samples, num_iterations, num_burn_in 

(array([0, 1, 2, 3, 4]),
 array([[0, 2],
        [1, 2],
        [2, 3],
        [1, 4]]),
 {0: Factor containing 5 variables
  -------------------------------------
  | X_0 X_1 X_2 X_3 X_4 | Probability |
  -------------------------------------
  |  0   0   0   0   0  |   0.0138889 |
  |  1   0   0   0   0  |   0.0277778 |
  |  0   1   0   0   0  |  0.00543478 |
  |  1   1   0   0   0  |   0.0362319 |
  |  0   0   1   0   0  |   0.0367647 |
  |  1   0   1   0   0  |  0.00490196 |
  |  0   1   1   0   0  |   0.0231481 |
  |  1   1   1   0   0  |   0.0185185 |
  |  0   0   2   0   0  |   0.0398936 |
  |  1   0   2   0   0  |  0.00177305 |
  |  0   1   2   0   0  |       0.035 |
  |  1   1   2   0   0  |  0.00666667 |
  |  0   0   0   1   0  |   0.0138889 |
  |  1   0   0   1   0  |   0.0277778 |
  |  0   1   0   1   0  |  0.00543478 |
  |  1   1   0   1   0  |   0.0362319 |
  |  0   0   1   1   0  |   0.0367647 |
  |  1   0   1   1   0  |  0.00490196 |
  |  0   1   1   1   0  |   0.0231

In [43]:
import pandas as pd
#ver 1: assuming factors

def _sample_step(nodes, factors, in_samples):
    """
    Performs gibbs sampling for a single iteration. Returns a sample for each node

    Args:
        nodes: numpy array of nodes
        factors: dictionary of factors e.g. factors[x1] returns the local factor for x1
        in_samples: dictionary of input samples (from previous iteration)

    Returns:
        dictionary of output samples where samples[x1] returns the sample for x1.
    """
    samples = copy.deepcopy(in_samples)

    """ YOUR CODE HERE """
    for node in nodes:
        currentVariable=node
        fixedVariablesDict={k:v for k,v in samples.items() if k!=currentVariable}
        #evidence is a problem
        currentFactor=factor_evidence(factors[currentVariable],fixedVariablesDict)
        variable_factor=factor_marginalize(currentFactor,np.setdiff1d(currentFactor.var, [currentVariable]) )
        # print(node,currentFactor.var, [currentVariable],np.setdiff1d(currentFactor.var, [currentVariable]) )
        proposal_distribution = variable_factor.val
        sample_space = np.arange(len(proposal_distribution))
        # Sample a value using the proposal distribution
        sampled_value = np.random.choice(sample_space, p=proposal_distribution)
        samples[node]=sampled_value
        #print(variable_factor,sampled_value,current_samples)

    """ END YOUR CODE HERE """

    return samples

def _get_conditional_probability(nodes, edges, factors, evidence, initial_samples, num_iterations, num_burn_in):
    """
    Returns the conditional probability p(Xf | Xe) where Xe is the set of observed nodes and Xf are the query nodes
    i.e. the unobserved nodes. The conditional probability is approximated using Gibbs sampling.

    Args:
        nodes: numpy array of nodes e.g. [x1, x2, ...].
        edges: numpy array of edges e.g. [i, j] implies that nodes[i] is the parent of nodes[j].
        factors: dictionary of Factors e.g. factors[x1] returns the conditional probability of x1 given all other nodes.
        evidence: dictionary of evidence e.g. evidence[x4] returns the provided evidence for x4.
        initial_samples: dictionary of initial samples to initialize Gibbs sampling.
        num_iterations: number of sampling iterations
        num_burn_in: number of burn-in iterations

    Returns:
        returns Factor of conditional probability.
    """
    assert num_iterations > num_burn_in
    conditional_prob = Factor()

    """ YOUR CODE HERE """
    print(nodes,edges)
    #update evidence
    evidencedFactorsDict=update_factor_dict_with_evidence(factors,evidence)
    evidencedFactorsDict
    # create children dict
    markovBlanketDict= {node:[node]+[child for parent,child in edges if parent==node] for node in nodes}
    markovBlanketSeperatedFactors={}
    # finding valid nodes post markov blankets treatment
    for node in markovBlanketDict.keys():
        factorsToMarginalizeOutList=np.setdiff1d(evidencedFactorsDict[node].var, markovBlanketDict[node])
        print(node,markovBlanketDict[node])
        markovBlanketSeperatedFactors[node]=factor_marginalize(evidencedFactorsDict[node],factorsToMarginalizeOutList)
        print(markovBlanketSeperatedFactors[node])

    start_samples=copy.deepcopy(initial_samples)
    for key in start_samples.keys():
        if evidence.get(key):
            start_samples[key]=evidence[key]

    burnedInSamples={}
    # num_iterations, num_burn_in
    for _ in tqdm(range(num_burn_in)):
        burnedInSamples=_sample_step(nodes=nodes, factors=markovBlanketSeperatedFactors, in_samples=start_samples)

    samplesList=[]
    for _ in tqdm(range(num_iterations)):
        samplesList.append(_sample_step(nodes=nodes, factors=markovBlanketSeperatedFactors, in_samples=burnedInSamples))
    df1=pd.DataFrame(samplesList)
    grouped = df1.groupby(list(df1.columns)).size().reset_index(name='counts')
    problemVal=np.zeros(len(node_factors[0].val))
    problemVal[grouped.apply(lambda x: assignment_to_index(x[df1.columns].to_numpy(),node_factors[0].card),axis=1)]=grouped.counts

    conditional_prob=Factor(var=factors[0].var,card=factors[0].card,val=problemVal/np.sum(problemVal))


    """ END YOUR CODE HERE """

    return conditional_prob

conditional_probability = _get_conditional_probability(nodes=nodes, edges=edges, factors=node_factors,
                                                           evidence=evidence, initial_samples=initial_samples,
                                                           num_iterations=num_iterations, num_burn_in=num_burn_in)

conditional_probability

[0 1 2 3 4] [[0 2]
 [1 2]
 [2 3]
 [1 4]]
0 [0, 2]
Factor containing 2 variables
-------------------------
| X_0 X_2 | Probability |
-------------------------
|  0   0  |   0.0772947 |
|  1   0  |    0.256039 |
|  0   1  |    0.239651 |
|  1   1  |   0.0936819 |
|  0   2  |    0.299574 |
|  1   2  |   0.0337589 |
-------------------------


1 [1, 2, 4]
Factor containing 3 variables
-----------------------------
| X_1 X_2 X_4 | Probability |
-----------------------------
|  0   0   0  |    0.161454 |
|  1   0   0  |  0.00521229 |
|  0   1   0  |    0.141151 |
|  1   1   0  |   0.0255158 |
|  0   2   0  |    0.112648 |
|  1   2   0  |   0.0540184 |
|  0   0   1  |   0.0562156 |
|  1   0   1  |    0.110451 |
|  0   1   1  |   0.0188852 |
|  1   1   1  |    0.147781 |
|  0   2   1  |  0.00609977 |
|  1   2   1  |    0.160567 |
-----------------------------


2 [2, 3]
Factor containing 2 variables
-------------------------
| X_2 X_3 | Probability |
-------------------------
|  0   0  |    0.

100%|██████████| 10000/10000 [00:05<00:00, 1732.76it/s]


Factor containing 5 variables
-------------------------------------
| X_0 X_1 X_2 X_3 X_4 | Probability |
-------------------------------------
|  0   0   0   0   0  |      0.0377 |
|  1   0   0   0   0  |      0.0049 |
|  0   1   0   0   0  |      0.0161 |
|  1   1   0   0   0  |      0.0017 |
|  0   0   1   0   0  |      0.0441 |
|  1   0   1   0   0  |      0.0051 |
|  0   1   1   0   0  |      0.0207 |
|  1   1   1   0   0  |       0.003 |
|  0   0   2   0   0  |      0.0918 |
|  1   0   2   0   0  |      0.0117 |
|  0   1   2   0   0  |      0.0467 |
|  1   1   2   0   0  |      0.0055 |
|  0   0   0   1   0  |      0.0382 |
|  1   0   0   1   0  |      0.0033 |
|  0   1   0   1   0  |      0.0165 |
|  1   1   0   1   0  |       0.002 |
|  0   0   1   1   0  |      0.0419 |
|  1   0   1   1   0  |      0.0054 |
|  0   1   1   1   0  |      0.0219 |
|  1   1   1   1   0  |      0.0024 |
|  0   0   2   1   0  |      0.0903 |
|  1   0   2   1   0  |      0.0113 |
|  0   1   2   1   0

In [6]:
import pandas as pd
#ver 2: assuming factors

def _sample_step(nodes, factors, in_samples):
    """
    Performs gibbs sampling for a single iteration. Returns a sample for each node

    Args:
        nodes: numpy array of nodes
        factors: dictionary of factors e.g. factors[x1] returns the local factor for x1
        in_samples: dictionary of input samples (from previous iteration)

    Returns:
        dictionary of output samples where samples[x1] returns the sample for x1.
    """
    samples = copy.deepcopy(in_samples)

    """ YOUR CODE HERE """
    for node in nodes:
        currentVariable=node
        fixedVariablesDict={k:v for k,v in samples.items() if k!=currentVariable}
        #evidence is a problem
        currentFactor=factor_evidence(factors[currentVariable],fixedVariablesDict)
        variable_factor=factor_marginalize(currentFactor,np.setdiff1d(currentFactor.var, [currentVariable]) )
        # print(node,currentFactor.var, [currentVariable],np.setdiff1d(currentFactor.var, [currentVariable]) )
        proposal_distribution = variable_factor.val
        sample_space = np.arange(len(proposal_distribution))
        # Sample a value using the proposal distribution
        sampled_value = np.random.choice(sample_space, p=proposal_distribution)
        samples[node]=sampled_value
        #print(variable_factor,sampled_value,current_samples)

    """ END YOUR CODE HERE """

    return samples

def _get_conditional_probability(nodes, edges, factors, evidence, initial_samples, num_iterations, num_burn_in):
    """
    Returns the conditional probability p(Xf | Xe) where Xe is the set of observed nodes and Xf are the query nodes
    i.e. the unobserved nodes. The conditional probability is approximated using Gibbs sampling.

    Args:
        nodes: numpy array of nodes e.g. [x1, x2, ...].
        edges: numpy array of edges e.g. [i, j] implies that nodes[i] is the parent of nodes[j].
        factors: dictionary of Factors e.g. factors[x1] returns the conditional probability of x1 given all other nodes.
        evidence: dictionary of evidence e.g. evidence[x4] returns the provided evidence for x4.
        initial_samples: dictionary of initial samples to initialize Gibbs sampling.
        num_iterations: number of sampling iterations
        num_burn_in: number of burn-in iterations

    Returns:
        returns Factor of conditional probability.
    """
    assert num_iterations > num_burn_in
    conditional_prob = Factor()

    """ YOUR CODE HERE """
    print(nodes,edges)
    #update evidence
    evidencedFactorsDict=update_factor_dict_with_evidence(factors,evidence)
    evidencedFactorsDict
    # create children dict
    markovBlanketDict= {node:[node]+[child for parent,child in edges if parent==node] for node in nodes}
    singleFactorsDict={}
    # marginalizing to their base prob
    for node in nodes:       
        factorsToMarginalizeOutList=np.setdiff1d(factors[node].var, [node])
        # print(node,markovBlanketDict[node])
        singleFactorsDict[node]=factor_marginalize(factors[node],factorsToMarginalizeOutList)
        #singleFactorsDict[node]=currentNodeFactor
        print(singleFactorsDict[node])
    
    nodes=[n for n in nodes if n not in evidence.keys()]

    markovSeperatedDict={}
    for node in nodes:
        markovJoinedVariablesList=markovBlanketDict[node] 
        print(node,markovJoinedVariablesList)
        currentMarkovFactor=compute_joint_distribution([evidencedFactorsDict[n] for n in markovJoinedVariablesList])
        factorsToMarginalizeOutList=np.setdiff1d(currentMarkovFactor.var, markovJoinedVariablesList)
        markovSeperatedDict[node]=factor_marginalize(currentMarkovFactor,factorsToMarginalizeOutList)
        print(markovSeperatedDict[node])
       
    # removing evidence from samping
    start_samples={k:v for k,v in copy.deepcopy(initial_samples).items() if k not in evidence.keys()}

    burnedInSamples={}
    # num_iterations, num_burn_in
    for _ in tqdm(range(num_burn_in)):
        burnedInSamples=_sample_step(nodes=nodes, factors=singleFactorsDict, in_samples=start_samples)

    samplesList=[]
    for _ in tqdm(range(num_iterations-num_burn_in)):
        samplesList.append(_sample_step(nodes=nodes, factors=singleFactorsDict, in_samples=burnedInSamples))
    
    #for sample in samplesList:
    #    for k,v in evidence.items():
    #        sample[k]=v

    df1=pd.DataFrame(samplesList)
    grouped = df1.groupby(list(df1.columns)).size().reset_index(name='counts')
    problemVal=np.zeros(len(node_factors[0].val))
    problemVal[grouped.apply(lambda x: assignment_to_index(x[df1.columns].to_numpy(),node_factors[0].card[nodes]),axis=1)]=grouped.counts

    conditional_prob=Factor(var=nodes,card=factors[0].card[nodes],val=problemVal/np.sum(problemVal))


    """ END YOUR CODE HERE """

    return conditional_prob

conditional_probability = _get_conditional_probability(nodes=nodes, edges=edges, factors=node_factors,
                                                           evidence=evidence, initial_samples=initial_samples,
                                                           num_iterations=num_iterations, num_burn_in=num_burn_in)

conditional_probability

[0 1 2 3 4] [[0 2]
 [1 2]
 [2 3]
 [1 4]]
Factor containing 1 variables
---------------------
| X_0 | Probability |
---------------------
|  0  |    0.616521 |
|  1  |    0.383479 |
---------------------


Factor containing 1 variables
---------------------
| X_1 | Probability |
---------------------
|  0  |    0.496454 |
|  1  |    0.503546 |
---------------------


Factor containing 1 variables
---------------------
| X_2 | Probability |
---------------------
|  0  |    0.405423 |
|  1  |    0.319595 |
|  2  |    0.274982 |
---------------------


Factor containing 1 variables
---------------------
| X_3 | Probability |
---------------------
|  0  |    0.496667 |
|  1  |    0.503333 |
---------------------


Factor containing 1 variables
---------------------
| X_4 | Probability |
---------------------
|  0  |       0.575 |
|  1  |       0.425 |
---------------------


0 [0, 2]
Factor containing 2 variables
-------------------------
| X_0 X_2 | Probability |
-------------------------


100%|██████████| 9000/9000 [00:03<00:00, 2676.83it/s]


Factor containing 4 variables
---------------------------------
| X_0 X_1 X_3 X_4 | Probability |
---------------------------------
|  0   0   0   0  |   0.0858889 |
|  1   0   0   0  |   0.0537778 |
|  0   1   0   0  |   0.0868889 |
|  1   1   0   0  |   0.0561111 |
|  0   0   1   0  |   0.0894444 |
|  1   0   1   0  |        0.06 |
|  0   1   1   0  |   0.0905556 |
|  1   1   1   0  |   0.0573333 |
|  0   0   0   1  |   0.0605556 |
|  1   0   0   1  |       0.041 |
|  0   1   0   1  |   0.0638889 |
|  1   1   0   1  |   0.0405556 |
|  0   0   1   1  |   0.0658889 |
|  1   0   1   1  |   0.0396667 |
|  0   1   1   1  |   0.0641111 |
|  1   1   1   1  |   0.0443333 |
---------------------------------


In [13]:
node_factors[0].card

array([2, 2, 3, 2, 2])

In [21]:
for n in [0,1,3,4]:
    print(np.where(node_factors[0].var==n))

(array([0]),)
(array([1]),)
(array([3]),)
(array([4]),)


In [22]:
np.where(np.in1d(node_factors[0].var, [0,1,3,4]))[0]

array([0, 1, 3, 4])

In [24]:
node_factors[0].card[[0,1,3,4]]

array([2, 2, 2, 2])

In [17]:
nodes

array([0, 1, 2, 3, 4])

In [4]:
#update evidence
evidencedFactorsDict=update_factor_dict_with_evidence(node_factors,evidence)
evidencedFactorsDict


{0: Factor containing 5 variables
 -------------------------------------
 | X_0 X_1 X_2 X_3 X_4 | Probability |
 -------------------------------------
 |  0   0   0   0   0  |           0 |
 |  1   0   0   0   0  |           0 |
 |  0   1   0   0   0  |           0 |
 |  1   1   0   0   0  |           0 |
 |  0   0   1   0   0  |   0.0367647 |
 |  1   0   1   0   0  |  0.00490196 |
 |  0   1   1   0   0  |   0.0231481 |
 |  1   1   1   0   0  |   0.0185185 |
 |  0   0   2   0   0  |           0 |
 |  1   0   2   0   0  |           0 |
 |  0   1   2   0   0  |           0 |
 |  1   1   2   0   0  |           0 |
 |  0   0   0   1   0  |           0 |
 |  1   0   0   1   0  |           0 |
 |  0   1   0   1   0  |           0 |
 |  1   1   0   1   0  |           0 |
 |  0   0   1   1   0  |   0.0367647 |
 |  1   0   1   1   0  |  0.00490196 |
 |  0   1   1   1   0  |   0.0231481 |
 |  1   1   1   1   0  |   0.0185185 |
 |  0   0   2   1   0  |           0 |
 |  1   0   2   1   0  |      

In [28]:
# create children dict
markovBlanketDict= {node:[node]+[child for parent,child in edges if parent==node] for node in nodes}
markovBlanketDict

{0: [0, 2], 1: [1, 2, 4], 2: [2, 3], 3: [3], 4: [4]}

In [30]:
markovBlanketSeperatedFactors={}
for node in evidencedFactorsDict.keys():
    factorsToMarginalizeOutList=np.setdiff1d(evidencedFactorsDict[node].var, markovBlanketDict[node])
    print(markovBlanketDict[node])
    markovBlanketSeperatedFactors[node]=factor_marginalize(evidencedFactorsDict[node],factorsToMarginalizeOutList)
markovBlanketSeperatedFactors

[0, 2]
[1, 2, 4]
[2, 3]
[3]
[4]


{0: Factor containing 2 variables
 -------------------------
 | X_0 X_2 | Probability |
 -------------------------
 |  0   0  |           0 |
 |  1   0  |           0 |
 |  0   1  |    0.718954 |
 |  1   1  |    0.281046 |
 |  0   2  |           0 |
 |  1   2  |           0 |
 -------------------------
 ,
 1: Factor containing 3 variables
 -----------------------------
 | X_1 X_2 X_4 | Probability |
 -----------------------------
 |  0   0   0  |           0 |
 |  1   0   0  |           0 |
 |  0   1   0  |    0.423453 |
 |  1   1   0  |   0.0765474 |
 |  0   2   0  |           0 |
 |  1   2   0  |           0 |
 |  0   0   1  |           0 |
 |  1   0   1  |           0 |
 |  0   1   1  |   0.0566556 |
 |  1   1   1  |    0.443344 |
 |  0   2   1  |           0 |
 |  1   2   1  |           0 |
 -----------------------------
 ,
 2: Factor containing 2 variables
 -------------------------
 | X_2 X_3 | Probability |
 -------------------------
 |  0   0  |           0 |
 |  1   0  |    0.

In [58]:
markovBlanketSeperatedFactors[0],factor_marginalize(markovBlanketSeperatedFactors[0],[3])

(Factor containing 2 variables
 -------------------------
 | X_0 X_2 | Probability |
 -------------------------
 |  0   0  |           0 |
 |  1   0  |           0 |
 |  0   1  |    0.718954 |
 |  1   1  |    0.281046 |
 |  0   2  |           0 |
 |  1   2  |           0 |
 -------------------------
 ,
 Factor containing 2 variables
 -------------------------
 | X_0 X_2 | Probability |
 -------------------------
 |  0   0  |           0 |
 |  1   0  |           0 |
 |  0   1  |    0.718954 |
 |  1   1  |    0.281046 |
 |  0   2  |           0 |
 |  1   2  |           0 |
 -------------------------
 )

In [32]:
nodes,initial_samples

(array([0, 1, 2, 3, 4]), {0: 0, 1: 0, 2: 0, 3: 0, 4: 0})

In [51]:
currentVariable=nodes[0]
fixedVariablesDict={k:v for k,v in initial_samples.items() if k!=currentVariable}
for key in fixedVariablesDict.keys():
    if evidence.get(key):
        fixedVariablesDict[key]=evidence[key]
#evidence is a problem
currentFactor=factor_evidence(markovBlanketSeperatedFactors[currentVariable],fixedVariablesDict)
factor_marginalize(currentFactor,np.setdiff1d(currentFactor.var, [currentVariable]) )

Factor containing 1 variables
---------------------
| X_0 | Probability |
---------------------
|  0  |    0.718954 |
|  1  |    0.281046 |
---------------------


In [77]:

current_samples=copy.deepcopy(initial_samples)
for key in current_samples.keys():
    if evidence.get(key):
        current_samples[key]=evidence[key]

for node in nodes:
    currentVariable=node
    fixedVariablesDict={k:v for k,v in current_samples.items() if k!=currentVariable}
    #evidence is a problem
    currentFactor=factor_evidence(markovBlanketSeperatedFactors[currentVariable],fixedVariablesDict)
    variable_factor=factor_marginalize(currentFactor,np.setdiff1d(currentFactor.var, [currentVariable]) )
    print(node,currentFactor.var, [currentVariable],np.setdiff1d(currentFactor.var, [currentVariable]) )
    proposal_distribution = variable_factor.val
    sample_space = np.arange(len(proposal_distribution))
    # Sample a value using the proposal distribution
    sampled_value = np.random.choice(sample_space, p=proposal_distribution)
    current_samples[node]=sampled_value
    print(variable_factor,sampled_value,current_samples)

0 [0 2] [0] [2]
Factor containing 1 variables
---------------------
| X_0 | Probability |
---------------------
|  0  |    0.718954 |
|  1  |    0.281046 |
---------------------

 0 {0: 0, 1: 0, 2: 1, 3: 0, 4: 0}
1 [1 2 4] [1] [2 4]
Factor containing 1 variables
---------------------
| X_1 | Probability |
---------------------
|  0  |    0.846905 |
|  1  |    0.153095 |
---------------------

 0 {0: 0, 1: 0, 2: 1, 3: 0, 4: 0}
2 [2 3] [2] [3]
Factor containing 1 variables
---------------------
| X_2 | Probability |
---------------------
|  0  |           0 |
|  1  |           1 |
|  2  |           0 |
---------------------

 1 {0: 0, 1: 0, 2: 1, 3: 0, 4: 0}
3 [3] [3] []
Factor containing 1 variables
---------------------
| X_3 | Probability |
---------------------
|  0  |         0.4 |
|  1  |         0.6 |
---------------------

 1 {0: 0, 1: 0, 2: 1, 3: 1, 4: 0}
4 [4] [4] []
Factor containing 1 variables
---------------------
| X_4 | Probability |
---------------------
|  0  |       0.

In [3]:
#update evidence
evidencedFactorsDict=update_factor_dict_with_evidence(node_factors,evidence)
evidencedFactorsDict
# create children dict
markovBlanketDict= {node:[node]+[child for parent,child in edges if parent==node] for node in nodes}
markovBlanketSeperatedFactors={}
# finding valid nodes post markov blankets treatment
for node in evidencedFactorsDict.keys():
    factorsToMarginalizeOutList=np.setdiff1d(evidencedFactorsDict[node].var, markovBlanketDict[node])
    # print(markovBlanketDict[node])
    markovBlanketSeperatedFactors[node]=factor_marginalize(evidencedFactorsDict[node],factorsToMarginalizeOutList)


def _sample_step(nodes, factors, in_samples):
    """
    Performs gibbs sampling for a single iteration. Returns a sample for each node

    Args:
        nodes: numpy array of nodes
        factors: dictionary of factors e.g. factors[x1] returns the local factor for x1
        in_samples: dictionary of input samples (from previous iteration)

    Returns:
        dictionary of output samples where samples[x1] returns the sample for x1.
    """
    samples = copy.deepcopy(in_samples)

    """ YOUR CODE HERE """
    for node in nodes:
        currentVariable=node
        fixedVariablesDict={k:v for k,v in samples.items() if k!=currentVariable}
        #evidence is a problem
        currentFactor=factor_evidence(factors[currentVariable],fixedVariablesDict)
        variable_factor=factor_marginalize(currentFactor,np.setdiff1d(currentFactor.var, [currentVariable]) )
        # print(node,currentFactor.var, [currentVariable],np.setdiff1d(currentFactor.var, [currentVariable]) )
        proposal_distribution = variable_factor.val
        sample_space = np.arange(len(proposal_distribution))
        # Sample a value using the proposal distribution
        sampled_value = np.random.choice(sample_space, p=proposal_distribution)
        samples[node]=sampled_value
        #print(variable_factor,sampled_value,current_samples)

    """ END YOUR CODE HERE """

    return samples

start_samples=copy.deepcopy(initial_samples)
for key in start_samples.keys():
    if evidence.get(key):
        start_samples[key]=evidence[key]

burnedInSamples={}
# num_iterations, num_burn_in
for _ in tqdm(range(num_burn_in)):
    burnedInSamples=_sample_step(nodes=nodes, factors=markovBlanketSeperatedFactors, in_samples=start_samples)

samplesList=[]
for _ in tqdm(range(num_iterations)):
    samplesList.append(_sample_step(nodes=nodes, factors=markovBlanketSeperatedFactors, in_samples=burnedInSamples))

100%|██████████| 10000/10000 [00:06<00:00, 1627.95it/s]


In [11]:
import pandas as pd 
df1=pd.DataFrame(samplesList)

df1.describe()

,0,1,2,3,4
count,10000.000000,10000.000000,10000.0,10000.000000,10000.00000
mean,0.282700,0.886200,1.0,0.594500,0.42410
std,0.450334,0.317584,0.0,0.491013,0.49423
min,0.000000,0.000000,1.0,0.000000,0.00000
25%,0.000000,1.000000,1.0,0.000000,0.00000
50%,0.000000,1.000000,1.0,1.000000,0.00000
75%,1.000000,1.000000,1.0,1.000000,1.00000
max,1.000000,1.000000,1.0,1.000000,1.00000


In [111]:
df1.mean()

0    0.9412
1    0.8022
2    0.7133
3    0.7061
4    0.2490
5    0.4173
6    0.6942
7    0.6245
8    0.5198
9    0.5554
dtype: float64

In [5]:
# finding unique counts of patterns
df1=pd.DataFrame(samplesList)
grouped = df1.groupby(list(df1.columns)).size().reset_index(name='counts')

grouped

,0,1,2,3,4,counts
0,0,0,1,0,0,179
1,0,0,1,0,1,156
2,0,0,1,1,0,270
3,0,0,1,1,1,223
4,0,1,1,0,0,1500
5,0,1,1,0,1,1068
6,0,1,1,1,0,2185
7,0,1,1,1,1,1592
8,1,0,1,0,0,71
9,1,0,1,0,1,60


In [6]:
# finding index of assignment
grouped.apply(lambda x: assignment_to_index(x[df1.columns].to_numpy(),node_factors[0].card),axis=1)

0      4
1     28
2     16
3     40
4      6
5     30
6     18
7     42
8      5
9     29
10    17
11    41
12     7
13    31
14    19
15    43
dtype: int64

In [7]:
df1=pd.DataFrame(samplesList)
grouped = df1.groupby(list(df1.columns)).size().reset_index(name='counts')
problemVal=np.zeros(len(node_factors[0].val))
problemVal[grouped.apply(lambda x: assignment_to_index(x[df1.columns].to_numpy(),node_factors[0].card),axis=1)]=grouped.counts
problemVal

array([   0.,    0.,    0.,    0.,  179.,   71., 1500.,  573.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,  270.,  115.,
       2185.,  866.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,  156.,   60., 1068.,  448.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,  223.,   64., 1592.,  630.,    0.,
          0.,    0.,    0.])

In [8]:
testFactor=Factor(var=node_factors[0].var,card=node_factors[0].card,val=problemVal/np.sum(problemVal))
testFactor

Factor containing 5 variables
-------------------------------------
| X_0 X_1 X_2 X_3 X_4 | Probability |
-------------------------------------
|  0   0   0   0   0  |           0 |
|  1   0   0   0   0  |           0 |
|  0   1   0   0   0  |           0 |
|  1   1   0   0   0  |           0 |
|  0   0   1   0   0  |      0.0179 |
|  1   0   1   0   0  |      0.0071 |
|  0   1   1   0   0  |        0.15 |
|  1   1   1   0   0  |      0.0573 |
|  0   0   2   0   0  |           0 |
|  1   0   2   0   0  |           0 |
|  0   1   2   0   0  |           0 |
|  1   1   2   0   0  |           0 |
|  0   0   0   1   0  |           0 |
|  1   0   0   1   0  |           0 |
|  0   1   0   1   0  |           0 |
|  1   1   0   1   0  |           0 |
|  0   0   1   1   0  |       0.027 |
|  1   0   1   1   0  |      0.0115 |
|  0   1   1   1   0  |      0.2185 |
|  1   1   1   1   0  |      0.0866 |
|  0   0   2   1   0  |           0 |
|  1   0   2   1   0  |           0 |
|  0   1   2   1   0

In [10]:
factor_marginalize(factor_evidence(testFactor,{2:1}),[2])

Factor containing 4 variables
---------------------------------
| X_0 X_1 X_3 X_4 | Probability |
---------------------------------
|  0   0   0   0  |      0.0179 |
|  1   0   0   0  |      0.0071 |
|  0   1   0   0  |        0.15 |
|  1   1   0   0  |      0.0573 |
|  0   0   1   0  |       0.027 |
|  1   0   1   0  |      0.0115 |
|  0   1   1   0  |      0.2185 |
|  1   1   1   0  |      0.0866 |
|  0   0   0   1  |      0.0156 |
|  1   0   0   1  |       0.006 |
|  0   1   0   1  |      0.1068 |
|  1   1   0   1  |      0.0448 |
|  0   0   1   1  |      0.0223 |
|  1   0   1   1  |      0.0064 |
|  0   1   1   1  |      0.1592 |
|  1   1   1   1  |       0.063 |
---------------------------------


In [121]:
grouped[df1.columns].iloc[0].to_numpy(),node_factors[0].card,assignment_to_index(grouped[df1.columns].iloc[0].to_numpy(),node_factors[0].card)

(array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1]),
 array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
 640)

In [131]:
node_factors[0].card,evidence

(array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2]), {})

In [106]:
for c in df1.columns:
    print(df1[c].unique())

[1 0]
[1 0]
[1 0]
[0 1]
[0 1]
[0 1]
[1 0]
[1 0]
[1 0]
[1 0]


In [103]:
{k:v.var for k,v in markovBlanketSeperatedFactors.items()}

{0: array([0, 1, 2]),
 1: array([1, 3]),
 2: array([2, 3, 4]),
 3: array([3, 5, 6]),
 4: array([4, 7, 8]),
 5: array([5]),
 6: array([6, 9]),
 7: array([7, 8]),
 8: array([8, 9]),
 9: array([9])}

In [107]:
markovBlanketSeperatedFactors

{0: Factor containing 3 variables
 -----------------------------
 | X_0 X_1 X_2 | Probability |
 -----------------------------
 |  0   0   0  |   0.0241935 |
 |  1   0   0  |    0.225806 |
 |  0   1   0  |        0.05 |
 |  1   1   0  |         0.2 |
 |  0   0   1  |  0.00652174 |
 |  1   0   1  |    0.243478 |
 |  0   1   1  |   0.0147059 |
 |  1   1   1  |    0.235294 |
 -----------------------------
 ,
 1: Factor containing 2 variables
 -------------------------
 | X_1 X_3 | Probability |
 -------------------------
 |  0   0  |   0.0970826 |
 |  1   0  |    0.402917 |
 |  0   1  |    0.290355 |
 |  1   1  |    0.209645 |
 -------------------------
 ,
 2: Factor containing 3 variables
 -----------------------------
 | X_2 X_3 X_4 | Probability |
 -----------------------------
 |  0   0   0  |   0.0737932 |
 |  1   0   0  |    0.176207 |
 |  0   1   0  |   0.0859863 |
 |  1   1   0  |    0.164014 |
 |  0   0   1  |    0.028281 |
 |  1   0   1  |    0.221719 |
 |  0   1   1  |    0.036

In [ ]:
def _get_conditional_probability(nodes, edges, factors, evidence, initial_samples, num_iterations, num_burn_in):
    # Initialize sample count dictionary
    sample_counts = defaultdict(int)

    # Start with initial samples
    current_samples = initial_samples

    # Perform Gibbs sampling
    for _ in range(num_iterations):
        # Get a new sample
        current_samples = _sample_step(nodes, factors, current_samples)

        # We don't count the samples in the burn-in phase
        if _ > num_burn_in:
            # Convert the current sample to an index or a hashable state
            state_index = assignment_to_index(current_samples, nodes)
            sample_counts[state_index] += 1

    # Now calculate probabilities from counts
    total_counts = sum(sample_counts.values())
    probabilities = {state: count / total_counts for state, count in sample_counts.items()}

    # Convert to a Factor (assuming a function to convert a dictionary to a Factor exists)
    conditional_prob = dict_to_factor(probabilities, nodes)

    return conditional_prob


In [95]:
import numpy as np

# Assuming `data` is a 10000x5 NumPy array with binary observations (1 or 0)
# Columns are A, B, C, D, E in order
data = np.random.randint(0, 2, size=(10000, 5))

# Calculate P(A)
# This is the number of times A occurs (is 1) divided by the total number of observations
p_A = np.mean(data[:, 0])

# Calculate P(A|B)
# This is the number of times both A and B occur together divided by the number of times B occurs
p_A_given_B = np.mean(data[:, 0][data[:, 1] == 1])

# Calculate P(C|D,E)
# This is the number of times C occurs with both D and E occurring divided by the number of times D and E both occur
mask_DE = np.logical_and(data[:, 3] == 1, data[:, 4] == 1)  # Create a mask where both D and E are 1
p_C_given_DE = np.mean(data[:, 2][mask_DE])

print(f"P(A) = {p_A}")
print(f"P(A|B) = {p_A_given_B}")
print(f"P(C|D,E) = {p_C_given_DE}")


P(A) = 0.5
P(A|B) = 0.5003923107100824
P(C|D,E) = 0.48519269776876267
